# VyOSの設定

## ネットワークを追加する

[vyos](http://wiki.vyos-users.jp/)には3つのネットワークを繋ぐが、docker-composeの挙動により、default以外のネットワークがコンテナにアタッチされる順番が保証されず、ネットワークとinterfaceの対応関係が固定されない（順序を固定する方法が見つからなかった）。そこで、wireのアタッチをコンテナ起動後に追加設定として行う。

因みに、docker createして、ネットワークをアタッチして、startさせる場合は、bridgeネットワークの名前順で認識される。dokcerの使われ方の主流では、デバイスとか気にしない感じで問題ないので、この辺の作りがイマイチなままなんだろうな。

vyosのinterfaceを確認する。起動時にアタッチされた2つ（eth0、eth1）が見えている。

In [ ]:
!sudo docker exec vyos ifconfig
!test $(sudo docker exec vyos ifconfig | grep eth0 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth1 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth2 | wc -l) -eq 0

dockerがコンテナ用に持っているネットワークを確認する。docker-composeで作ったネットワークには起動場所の接頭語が付くので注意。

In [ ]:
!sudo docker network ls

vyosにroot_wireをアタッチ。

In [ ]:
!sudo docker network connect root_wire vyos

3つ目のネットワークがeth2として増えているのを確認する。

In [ ]:
!sudo docker exec vyos ifconfig
!test $(sudo docker exec vyos ifconfig | grep eth0 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth1 | wc -l) -eq 1
!test $(sudo docker exec vyos ifconfig | grep eth2 | wc -l) -eq 1

## Ansibleで設定を行う

vyosのルータ設定はansible経由で行う。connection:dockerで接続するため、vyos_configが使えないので、スクリプトファイルをコピーしてシェル実行する。併せて、docker execからvyosのコマンドを叩くためのスクリプトファイルもコピーしておく。

[vyosの設定](/edit/wifi/vyos-setup.vsh)

In [ ]:
!sudo ansible-playbook -i vyos, vyos-setup.yml -vvv

## VyOSコマンドラッパを確認する

[コマンドのラッパ](/edit/wifi/run-command.sh)

こんな感じでvyosのコマンドが叩ける。

In [ ]:
!sudo docker exec --user vyos vyos /home/vyos/run-command.sh show interfaces

因みに、VyOSは実質Debianなので、interfaceが絡んだ機能（例えば、FireWallやQoS）は、OSのネットワーク設定で実現されている。このため、本事例のようにコンテナで利用すると、ネットワークデバイスが完全仮想化されていないことから、ネットワーク関連の機能のいくつかが動作しない。vyosにdocker execしてドヤ顔するのが不要で、そこそこのスペックのマシンが用意できるなら、普通にxenとか使った完全仮想化の方が、トラブルも少ないだろう。